
# 网络参数

[![下载Notebook](https://gitee.com/mindspore/docs/raw/master/resource/_static/logo_notebook.png)](https://obs.dualstack.cn-north-4.myhuaweicloud.com/mindspore-website/notebook/tutorials-develop/tutorials/zh_cn/mindspore_parameter.ipynb)&emsp;[![下载样例代码](https://gitee.com/mindspore/docs/raw/master/resource/_static/logo_download_code.png)](https://obs.dualstack.cn-north-4.myhuaweicloud.com/mindspore-website/notebook/tutorials-develop/tutorials/zh_cn/mindspore_parameter.py)&emsp;[![查看源文件](https://gitee.com/mindspore/docs/raw/master/resource/_static/logo_source.png)](https://gitee.com/mindspore/docs/blob/tutorials-develop/tutorials/source_zh_cn/intermediate/build_net/parameter.ipynb)

## 概述

MindSpore提供了网络参数初始化模块，用户可以通过封装算子来调用字符串、Initializer子类或自定义Tensor等方式完成对网络参数进行初始化。本章主要介绍变量张量`Parameter`的、变量张量元组`ParameterTuple`，网络的初始化方法和网络参数更新。

## 变量张量（[Parameter](https://www.mindspore.cn/docs/api/zh-CN/master/api_python/mindspore/mindspore.Parameter.html#mindspore.Parameter)）

变量张量`Parameter`，表示在训练网络时需要被更新的参数，一般包括权重`weight`和偏置`bias`。

### 初始化

变量张量`Parameter`的初始化方法有多种。变量张量`default_input`为输入数据，支持传入`Tensor`、`Initializer`、`int`和`float`四种数据类型；`name`为可设置变量张量的名称；`requires_grad`表示在网络训练过程，是否需要计算参数梯度，如果不需要计算参数梯度，将`requires_grad`设置为`False`。

+ 根据`int`或`float`型数据直接创建。

In [91]:
from mindspore import Parameter

x = Parameter(default_input=2.0, name='x')
y = Parameter(default_input=5.0, name='y')
z = Parameter(default_input=5, name='z', requires_grad=False)

print(type(x))
print(x, "value:", x.asnumpy())
print(y, "value:", y.asnumpy())
print(z, "value:", z.asnumpy())

<class 'mindspore.common.parameter.ParameterTensor'>
Parameter (name=x, shape=(), dtype=Float32, requires_grad=True) value: 2.0
Parameter (name=y, shape=(), dtype=Float32, requires_grad=True) value: 5.0
Parameter (name=z, shape=(), dtype=Int32, requires_grad=False) value: 5


+ 根据`Tensor`创建。

In [18]:
import numpy as np
from mindspore import Tensor

x = Parameter(default_input=Tensor(np.arange(2 * 3).reshape((2, 3))), name="x")
print(x)

Parameter (name=x, shape=(2, 3), dtype=Int64, requires_grad=True)


+ 使用`Initializer`创建。

In [22]:
from mindspore.common.initializer import initializer
from mindspore import dtype as mstype

x = Parameter(default_input=initializer('ones', [1, 2, 3], mstype.float32), name='x')
print(x)

Parameter (name=x, shape=(1, 2, 3), dtype=Float32, requires_grad=True)


### 属性

变量张量`Parameter`的默认属性有变量名称`name`、形状`shape`、数据类型`dtype`和是否需要进行求导`requires_grad`。其他属性如下所示：

+ `data`： 变量张量`Parameter`本身。

+ `sliced`：用在自动并行场景下，表示变量张量`Parameter`里保存的数据是否是分片数据。如果是，就不再对其进行切分，如果不是，需要根据网络并行策略确认是否对其进行切分。

+ `layerwise_parallel`：是否支持layerwise并行。如果支持，参数就不会进行广播和梯度聚合，反之则需要。

下例通过`Tensor`初始化一个变量张量`Parameter`，并获取变量张量`Parameter`的相关属性。示例如下：

In [141]:
x = Parameter(default_input=Tensor(np.arange(2 * 3).reshape((2, 3))), name="x")

print("x: ", x)
print("x.data: ", x.data)
print("sliced: ", x.sliced)
print("layerwise_parallel: ", x.layerwise_parallel)

x:  Parameter (name=x, shape=(2, 3), dtype=Int64, requires_grad=True)
x.data:  Parameter (name=x, shape=(2, 3), dtype=Int64, requires_grad=True)
sliced:  False
layerwise_parallel:  False


### 方法

+ `clone`：克隆变量张量`Parameter`，克隆完成后可以给新的变量张量`Parameter`指定新的名称。

In [48]:
x = Parameter(default_input=initializer('ones', [1, 2, 3], mstype.float32))
x_clone = x.clone()
x_clone.name = "x_clone"

print(x)
print(x_clone)

Parameter (name=Parameter, shape=(1, 2, 3), dtype=Float32, requires_grad=True)
Parameter (name=x_clone, shape=(1, 2, 3), dtype=Float32, requires_grad=True)


+ `set_data`：修改变量张量`Parameter`的数据或形状`shape`，支持传入`Tensor`、`Initializer`、`int`和`float`四种数据类型。`set_data`方法有`data`和`slice_shape`两种入参。其中`data`表示变量张量`Parameter`新传入的数据；`slice_shape`表示是否修改变量张量`Parameter`的形状`shape`，默认为False。

In [136]:
x = Parameter(Tensor(np.ones((1, 2)), mindspore.float32), name="x", requires_grad=True)
print(x, x.asnumpy())

y = x.set_data(Tensor(np.zeros((1, 2)), mindspore.float32))
print(y, y.asnumpy())

z = x.set_data(Tensor(np.ones((1, 4)), mindspore.float32), slice_shape=True)
print(z, z.asnumpy())

Parameter (name=x, shape=(1, 2), dtype=Float32, requires_grad=True) [[1. 1.]]
Parameter (name=x, shape=(1, 2), dtype=Float32, requires_grad=True) [[0. 0.]]
Parameter (name=x, shape=(1, 4), dtype=Float32, requires_grad=True) [[1. 1. 1. 1.]]


+ `init_data`：并行场景下存在参数的形状发生变化的情况，用户可以调用`Parameter`的`init_data`方法得到原始数据。

In [144]:
x = Parameter(Tensor(np.ones((1, 2)), mindspore.float32), name="x", requires_grad=True)
print(x.init_data(), x.init_data().asnumpy())

Parameter (name=x, shape=(1, 2), dtype=Float32, requires_grad=True) [[1. 1.]]


## 变量张量元组（[ParameterTuple](https://www.mindspore.cn/docs/api/zh-CN/master/api_python/mindspore/mindspore.ParameterTuple.html#mindspore.ParameterTuple)）

变量张量元组`ParameterTuple`用于保存多个`Parameter`，继承于元组`tuple`，提供克隆功能。如下示例提供`ParameterTuple`创建方法：

In [102]:
import numpy as np
from mindspore import Tensor, Parameter, ParameterTuple
from mindspore import dtype as mstype
from mindspore.common.initializer import initializer

# 创建
x = Parameter(default_input=Tensor(np.arange(2 * 3).reshape((2, 3))), name="x")
y = Parameter(default_input=initializer('ones', [1, 2, 3], mstype.float32), name='y')
z = Parameter(default_input=2.0, name='z')
params = ParameterTuple((x, y, z))

# 从params克隆并修改名称为"params_copy"
params_copy = params.clone("params_copy")
print(params, "\n")
print(params_copy)

(Parameter (name=x, shape=(2, 3), dtype=Int64, requires_grad=True), Parameter (name=y, shape=(1, 2, 3), dtype=Float32, requires_grad=True), Parameter (name=z, shape=(), dtype=Float32, requires_grad=True)) 

(Parameter (name=params_copy.x, shape=(2, 3), dtype=Int64, requires_grad=True), Parameter (name=params_copy.y, shape=(1, 2, 3), dtype=Float32, requires_grad=True), Parameter (name=params_copy.z, shape=(), dtype=Float32, requires_grad=True))


## 网络参数初始化

MindSpore提供了多种网络参数初始化的方式，并在部分算子中封装了参数初始化的功能。本节以`Conv2d`算子为例，分别介绍使用`Initializer`子类，字符串和自定义`Tensor`等方式对网络中的参数进行初始化。

### Initializer子类初始化

使用`Initializer`子类对网络参数进行初始化，如要使用`Initializer`子类中的参数，需使用`Initializer`子类的方式对参数进行初始化，示例如下：

In [ ]:
import numpy as np
import mindspore.nn as nn
from mindspore import Tensor
from mindspore import set_seed
from mindspore.common.initializer import Normal

set_seed(1)

input_data = Tensor(np.ones([1, 3, 16, 50], dtype=np.float32))
net = nn.Conv2d(3, 64, 3, weight_init=Normal(0.2))
output = net(input_data)
print(output)

```text
[[[[ 6.2076533e-01  8.7720710e-01  8.7720710e-01 ...  8.7720710e-01
     8.7720710e-01  2.7743810e-01]
   [ 6.5210247e-01  7.0859784e-01  7.0859784e-01 ...  7.0859784e-01
     7.0859784e-01 -1.1080378e-01]
   [ 6.5210247e-01  7.0859784e-01  7.0859784e-01 ...  7.0859784e-01
     7.0859784e-01 -1.1080378e-01]
     ...
   [ 1.1884158e+00  5.6527245e-01  5.6527245e-01 ...  5.6527245e-01
     5.6527245e-01 -6.5525830e-01]
   [ 1.1884158e+00  5.6527245e-01  5.6527245e-01 ...  5.6527245e-01
     5.6527245e-01 -6.5525824e-01]
   [ 1.6852863e+00  1.0636344e+00  1.0636344e+00 ...  1.0636345e+00
     1.0636345e+00 -9.2076123e-02]]]]
```

### 字符串初始化

使用字符串对网络参数进行初始化，字符串的内容需要与`Initializer`子类的名称保持一致(字母不区分大小写)，使用字符串方式进行初始化将使用`Initializer`子类中的默认参数，例如使用字符串`Normal`等同于使用`Initializer`的子类`Normal()`，示例如下：


In [ ]:
import numpy as np
import mindspore.nn as nn
from mindspore import Tensor
from mindspore import set_seed

set_seed(1)

input_data = Tensor(np.ones([1, 3, 16, 50], dtype=np.float32))
net = nn.Conv2d(3, 64, 3, weight_init='Normal')
output = net(input_data)
print(output)

```text
[[[[ 3.10382620e-02  4.38603461e-02  4.38603461e-02 ...  4.38603461e-02
     4.38603461e-02  1.38719045e-02]
   [ 3.26051228e-02  3.54298912e-02  3.54298912e-02 ...  3.54298949e-02
     3.54298949e-02 -5.54019213e-03]
   [ 3.26051228e-02  3.54298912e-02  3.54298912e-02 ...  3.54298912e-02
     3.54298912e-02 -5.54019120e-03]
   ...
   [ 4.38403059e-03 -3.60766202e-02 -3.60766202e-02 ... -3.60766277e-02
    -3.60766277e-02 -2.95619294e-02]
   [ 4.38403059e-03 -3.60766202e-02 -3.60766202e-02 ... -3.60766202e-02
    -3.60766202e-02 -2.95619294e-02]
   [ 1.33139016e-02  6.74417242e-05  6.74417242e-05 ...  6.74389303e-05
     6.74389303e-05 -2.27325857e-02]]]]
```

### 自定义的Tensor

用户也可以通过自定义`Tensor`的方式来对参数进行初始化，示例如下：

In [ ]:
import numpy as np
import mindspore.nn as nn
from mindspore import Tensor
from mindspore import dtype as mstype

weight = Tensor(np.ones([64, 3, 3, 3]), dtype=mstype.float32)
input_data = Tensor(np.ones([1, 3, 16, 50], dtype=np.float32))
net = nn.Conv2d(3, 64, 3, weight_init=weight)
output = net(input_data)
print(output)

```text
[[[[12. 18. 18. ... 18. 18. 12.]
   [18. 27. 27. ... 27. 27. 18.]
   [18. 27. 27. ... 27. 27. 18.]
   ...
   [18. 27. 27. ... 27. 27. 18.]
   [18. 27. 27. ... 27. 27. 18.]
   [12. 18. 18. ... 18. 18. 12.]]]]
```

## 网络参数更新

MindSpore提供了网络参数更新功能，使用`nn.ParameterUpdate`可对网络参数进行更新，其输入的参数类型必须为张量，且张量`shape`需要与原网络参数`shape`保持一致。更新网络的权重参数示例如下：

In [129]:
import numpy as np
from mindspore import dtype as mstype
from mindspore import nn, Tensor

# 构建网络
network = nn.Dense(3, 4)
# 获取网络的权重参数
param = network.parameters_dict()['weight']
print("param:\n", param.asnumpy())

# 更新权重参数
update = nn.ParameterUpdate(param)
weight = Tensor(np.arange(12).reshape((4, 3)), mstype.float32)
output = update(weight)
print("param update:\n", output)

param:
 [[-0.0077879  -0.01631169 -0.00774882]
 [ 0.00361533 -0.01483704  0.00744422]
 [ 0.00665934 -0.01925852 -0.02213076]
 [-0.00279421  0.01458707  0.00358556]]
param update:
 [[ 0.  1.  2.]
 [ 3.  4.  5.]
 [ 6.  7.  8.]
 [ 9. 10. 11.]]
